In [ ]:
from snownlp import SnowNLP
import pandas as pd
import json
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# 中文 agent 名称与英文的映射和对应的颜色
agent_name_map = {
    "蓝色思考帽(管理)": "Blue Hat (Management)",
    "绿色思考帽(创新)": "Green Hat (Creativity)",
    "白色思考帽(客观)": "White Hat (Objectivity)",
    "黄色思考帽(积极)": "Yellow Hat (Positivity)",
    "红色思考帽(情感)": "Red Hat (Emotion)",
    "黑色思考帽(批判)": "Black Hat (Criticism)"
}

# agent 对应的颜色
agent_colors = {
    "Blue Hat (Management)": "blue",
    "Green Hat (Creativity)": "green",
    "White Hat (Objectivity)": "lightgray",  # 使用灰白色代替白色
    "Yellow Hat (Positivity)": "yellow",
    "Red Hat (Emotion)": "red",
    "Black Hat (Criticism)": "black"
}

# 读取 conversation_history.json 文件
with open('/data1/dxw_data/llm/is_agents/agents/six_hats/task5/conversation_history.json', 'r', encoding='utf-8') as file:
    conversation_data = json.load(file)

# 将消息按 agent 分类
messages_data = []
for conversation in conversation_data:
    agent = conversation['agent']
    message = conversation['message']
    agent_english = agent_name_map.get(agent, agent)
    
    # 将 message 切分为句子，并对每个句子进行情感分析
    sentences = [sentence.strip() for sentence in message.split('。') if sentence.strip()]  # 使用句号分割
    if sentences:
        sentence_sentiments = [SnowNLP(sentence).sentiments for sentence in sentences]
        avg_sentiment = sum(sentence_sentiments) / len(sentence_sentiments)
    else:
        avg_sentiment = 0  # 如果 message 为空，则情感值设为 0
    
    # 将每个 message 的情感得分和 agent 颜色保存
    messages_data.append({'agent': agent_english, 'sentiment': avg_sentiment, 'color': agent_colors[agent_english]})

# 全局情感变化情况（按照消息顺序）
global_sentiments = pd.DataFrame(messages_data)

# 绘制全局情感变化趋势图，按消息顺序绘制，并为每个 message 设置颜色
plt.figure(figsize=(10, 6))

# 按顺序绘制每条 message 的情感分数，使用对应的颜色
for i in range(len(global_sentiments) - 1):
    # 画点
    plt.plot([i + 1], [global_sentiments.loc[i, 'sentiment']], marker='o', color=global_sentiments.loc[i, 'color'])
    
    # 连接线的颜色与后一个点的颜色一致
    plt.plot([i + 1, i + 2], 
             [global_sentiments.loc[i, 'sentiment'], global_sentiments.loc[i + 1, 'sentiment']], 
             color=global_sentiments.loc[i + 1, 'color'])

# 绘制最后一个点
plt.plot([len(global_sentiments)], [global_sentiments.iloc[-1]['sentiment']], marker='o', color=global_sentiments.iloc[-1]['color'])

plt.title('Global Sentiment Trend (Color by Agent)')
plt.xlabel('Message Order')
plt.ylabel('Sentiment Score')
plt.grid(True)

# 自定义图例，确保每个 agent 的颜色对应正确
legend_elements = [Line2D([0], [0], color=color, lw=2, label=agent) for agent, color in agent_colors.items()]
plt.legend(handles=legend_elements, loc='best')

# 保存全局情感变化趋势图
plt.savefig('/data1/dxw_data/llm/is_agents/agents/six_hats/task5/all-sentiment.png', dpi=300)

# 显示图像
plt.show()

# 绘制每个 agent 内部的情感变化趋势
plt.figure(figsize=(10, 6))
for agent, agent_data in global_sentiments.groupby('agent'):
    agent_sentiments = agent_data['sentiment'].tolist()
    plt.plot(range(1, len(agent_sentiments) + 1), agent_sentiments, marker='o', linestyle='-', color=agent_colors[agent], label=agent)

plt.title('Sentiment Trend for Each Agent')
plt.xlabel('Message Order within Agent')
plt.ylabel('Sentiment Score')
plt.legend(loc='best')
plt.grid(True)
plt.show()
